# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [24]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score 
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [25]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [26]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2177783363077,
 'week52high': 137.7,
 'week52low': 57.9,
 'week52change': 0.8473775089928776,
 'sharesOutstanding': 17035628718,
 'float': 0,
 'avg10Volume': 101557795,
 'avg30Volume': 99852359,
 'day200MovingAvg': 110.25,
 'day50MovingAvg': 122.51,
 'employees': 0,
 'ttmEPS': 3.39,
 'ttmDividendRate': 0.8383001270752216,
 'dividendYield': 0.006503352343904289,
 'nextDividendDate': '0',
 'exDividendDate': '2020-11-01',
 'nextEarningsDate': '0',
 'peRatio': 37.97809320810556,
 'beta': 1.203023764496749,
 'maxChangePercent': 48.809474141373,
 'year5ChangePercent': 4.005974836597889,
 'year2ChangePercent': 2.248150542402974,
 'year1ChangePercent': 0.8466572338095532,
 'ytdChangePercent': 0.7574506221717776,
 'month6ChangePercent': 0.46763862934128,
 'month3ChangePercent': 0.1488017911999971,
 'month1ChangePercent': 0.06444495864656297,
 'day30ChangePercent': 0.0636410299739902,
 'day5ChangePercent': 0.05075566063539871}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [27]:
data['year1ChangePercent']

0.8466572338095532

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [28]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [29]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ], index=my_columns), ignore_index = True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,120.190,0.413071,N/A
1,AAL,17.349,-0.410531,N/A
2,AAP,160.950,0.0583297,N/A
3,AAPL,129.811,0.8678,N/A
4,ABBV,105.680,0.237976,N/A
...,...,...,...,...
500,YUM,110.773,0.0855623,N/A
501,ZBH,156.730,0.000230118,N/A
502,ZBRA,378.480,0.492606,N/A
503,ZION,42.060,-0.154628,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [30]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,39.070,2.18807,N/A
1,345,NVDA,541.310,1.36476,N/A
2,31,AMD,97.847,1.3112,N/A
3,275,LB,40.261,1.21786,N/A
4,23,ALB,140.330,1.1389,N/A
5,387,PYPL,237.300,1.10746,N/A
6,339,NOW,563.350,0.945603,N/A
7,24,ALGN,526.980,0.928737,N/A
8,179,FCX,25.319,0.911223,N/A
9,3,AAPL,129.811,0.8678,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [31]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')
    
    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:50000000
50000000


In [32]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe['Price'][i])
final_dataframe

/Users/constantinechung/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,39.070,2.18807,25595
1,345,NVDA,541.310,1.36476,1847
2,31,AMD,97.847,1.3112,10220
3,275,LB,40.261,1.21786,24837
4,23,ALB,140.330,1.1389,7126
5,387,PYPL,237.300,1.10746,4214
6,339,NOW,563.350,0.945603,1775
7,24,ALGN,526.980,0.928737,1897
8,179,FCX,25.319,0.911223,39496
9,3,AAPL,129.811,0.8678,7703


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [55]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series([symbol, 
                       data[symbol]['quote']['latestPrice'],
                       'N/A',
                       data[symbol]['stats']['year1ChangePercent'],
                       'N/A',
                       data[symbol]['stats']['month6ChangePercent'],
                       'N/A',
                       data[symbol]['stats']['month3ChangePercent'],
                       'N/A',
                       data[symbol]['stats']['month1ChangePercent'],
                       'N/A',
                       'N/A'
                       ], index = hqm_columns), ignore_index = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,120.773,N/A,0.414871,N/A,0.356859,N/A,0.18092,N/A,0.0636684,N/A,N/A
1,AAL,16.946,N/A,-0.40949,N/A,-0.0100317,N/A,0.228865,N/A,0.330789,N/A,N/A
2,AAP,167.210,N/A,0.0602413,N/A,0.136154,N/A,0.0462302,N/A,0.0703239,N/A,N/A
3,AAPL,131.821,N/A,0.875243,N/A,0.470857,N/A,0.142473,N/A,0.0642111,N/A,N/A
4,ABBV,104.981,N/A,0.242408,N/A,0.119813,N/A,0.1828,N/A,0.0659081,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.760,N/A,0.0876638,N/A,0.184743,N/A,0.141517,N/A,0.0312565,N/A,N/A
501,ZBH,151.710,N/A,0.000223881,N/A,0.146233,N/A,0.0224619,N/A,-0.00936861,N/A,N/A
502,ZBRA,386.801,N/A,0.498091,N/A,0.420277,N/A,0.493187,N/A,0.0926394,N/A,N/A
503,ZION,42.242,N/A,-0.151637,N/A,0.157333,N/A,0.330123,N/A,0.0550359,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [56]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,120.773,N/A,0.414871,86.9307,0.356859,80.7921,0.18092,64.9505,0.0636684,72.6733,N/A
1,AAL,16.946,N/A,-0.40949,2.77228,-0.0100317,12.0792,0.228865,78.2178,0.330789,99.2079,N/A
2,AAP,167.210,N/A,0.0602413,47.7228,0.136154,37.4257,0.0462302,28.5149,0.0703239,75.2475,N/A
3,AAPL,131.821,N/A,0.875243,98.2178,0.470857,90.099,0.142473,54.8515,0.0642111,72.8713,N/A
4,ABBV,104.981,N/A,0.242408,73.2673,0.119813,33.8614,0.1828,65.3465,0.0659081,74.0594,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.760,N/A,0.0876638,51.8812,0.184743,49.901,0.141517,54.4554,0.0312565,58.4158,N/A
501,ZBH,151.710,N/A,0.000223881,38.2178,0.146233,39.802,0.0224619,22.5743,-0.00936861,34.2574,N/A
502,ZBRA,386.801,N/A,0.498091,91.2871,0.420277,86.5347,0.493187,97.2277,0.0926394,80.5941,N/A
503,ZION,42.242,N/A,-0.151637,16.4356,0.157333,41.5842,0.330123,90.8911,0.0550359,69.901,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [57]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,120.773,N/A,0.414871,86.9307,0.356859,80.7921,0.18092,64.9505,0.0636684,72.6733,76.3366
1,AAL,16.946,N/A,-0.40949,2.77228,-0.0100317,12.0792,0.228865,78.2178,0.330789,99.2079,48.0693
2,AAP,167.210,N/A,0.0602413,47.7228,0.136154,37.4257,0.0462302,28.5149,0.0703239,75.2475,47.2277
3,AAPL,131.821,N/A,0.875243,98.2178,0.470857,90.099,0.142473,54.8515,0.0642111,72.8713,79.0099
4,ABBV,104.981,N/A,0.242408,73.2673,0.119813,33.8614,0.1828,65.3465,0.0659081,74.0594,61.6337
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.760,N/A,0.0876638,51.8812,0.184743,49.901,0.141517,54.4554,0.0312565,58.4158,53.6634
501,ZBH,151.710,N/A,0.000223881,38.2178,0.146233,39.802,0.0224619,22.5743,-0.00936861,34.2574,33.7129
502,ZBRA,386.801,N/A,0.498091,91.2871,0.420277,86.5347,0.493187,97.2277,0.0926394,80.5941,88.9109
503,ZION,42.242,N/A,-0.151637,16.4356,0.157333,41.5842,0.330123,90.8911,0.0550359,69.901,54.703


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [58]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,24.730,N/A,0.894907,98.4158,1.31723,99.802,0.458221,95.8416,0.161443,90.297,96.0891
1,ALGN,536.670,N/A,0.931605,98.6139,0.982956,99.2079,0.589591,98.4158,0.130201,87.3267,95.8911
2,TWTR,56.200,N/A,0.760564,96.4356,0.565281,94.6535,0.365357,92.6733,0.270388,98.2178,95.495
3,XLNX,157.970,N/A,0.609123,94.6535,0.643036,96.4356,0.507584,97.4257,0.163338,90.6931,94.802
4,LRCX,492.200,N/A,0.793964,96.6337,0.635985,96.0396,0.639615,99.0099,0.130353,87.5248,94.802
5,AMAT,88.546,N/A,0.513358,91.6832,0.536895,93.8614,0.581626,98.2178,0.193423,94.0594,94.4554
6,AMD,100.000,N/A,1.30064,99.604,0.815709,98.6139,0.270512,83.5644,0.161695,90.495,93.0693
7,LB,39.000,N/A,1.22303,99.4059,1.39999,100,0.308829,87.7228,0.116109,84.7525,92.9703
8,PYPL,247.150,N/A,1.12185,99.2079,0.442194,88.1188,0.282745,85.3465,0.20163,95.0495,91.9307
9,ALXN,160.740,N/A,0.433905,88.3168,0.403345,84.5545,0.358701,92.4752,0.241943,97.0297,90.5941


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [59]:
portfolio_input()

Enter the value of your portfolio:10000000000


In [63]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

/Users/constantinechung/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,24.730,8087343,0.894907,98.4158,1.31723,99.802,0.458221,95.8416,0.161443,90.297,96.0891
1,ALGN,536.670,372668,0.931605,98.6139,0.982956,99.2079,0.589591,98.4158,0.130201,87.3267,95.8911
2,TWTR,56.200,3558718,0.760564,96.4356,0.565281,94.6535,0.365357,92.6733,0.270388,98.2178,95.495
3,XLNX,157.970,1266063,0.609123,94.6535,0.643036,96.4356,0.507584,97.4257,0.163338,90.6931,94.802
4,LRCX,492.200,406338,0.793964,96.6337,0.635985,96.0396,0.639615,99.0099,0.130353,87.5248,94.802
5,AMAT,88.546,2258712,0.513358,91.6832,0.536895,93.8614,0.581626,98.2178,0.193423,94.0594,94.4554
6,AMD,100.000,2000000,1.30064,99.604,0.815709,98.6139,0.270512,83.5644,0.161695,90.495,93.0693
7,LB,39.000,5128205,1.22303,99.4059,1.39999,100,0.308829,87.7228,0.116109,84.7525,92.9703
8,PYPL,247.150,809225,1.12185,99.2079,0.442194,88.1188,0.282745,85.3465,0.20163,95.0495,91.9307
9,ALXN,160.740,1244245,0.433905,88.3168,0.403345,84.5545,0.358701,92.4752,0.241943,97.0297,90.5941


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [64]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [65]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [66]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [67]:
writer.save()